In [1]:
import random

# Constants
n_days = 5
n_slots_per_day = 4
n_rooms = 5
n_sections = 10
n_professors = 4
n_courses = 4
courses_map = {0: 'Math', 1: 'Chemistry', 2: 'Physics', 3: 'Biology'}
teachers_map = {0: 'Mr. Smith', 1: 'Ms. Johnson', 2: 'Dr. Garcia', 3: 'Prof. Lee'}
sections_map = {i: f'Section {i}' for i in range(n_sections)}
days_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday'}
times_map = {0: '8:00 - 9:30', 1: '9:45 - 11:15', 2: '11:30 - 13:00', 3: '14:00 - 15:30'}

def generate_initial_timetable():
    timetable = [[[None for _ in range(n_rooms)] for _ in range(n_slots_per_day)] for _ in range(n_days)]
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            for room in range(n_rooms):
                if random.random() < 0.5:
                    section = random.randint(0, n_sections - 1)
                    course = random.randint(0, n_courses - 1)
                    professor = random.randint(0, n_professors - 1)
                    timetable[day][slot][room] = (section, course, professor)
    return timetable

def encode_timetable(timetable):
    encoded = ""
    for day in timetable:
        for slot in day:
            for session in slot:
                if session is None:
                    encoded += '0' * 8
                else:
                    section, course, professor = session
                    encoded += format(section, '04b') + format(course, '02b') + format(professor, '02b')
    return encoded

def decode_timetable(encoded):
    timetable = generate_initial_timetable()
    index = 0
    bits_per_session = 8
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            for room in range(n_rooms):
                session = encoded[index:index+8]
                if session != '0' * 8:
                    section = int(session[:4], 2)
                    course = int(session[4:6], 2)
                    professor = int(session[6:8], 2)
                    timetable[day][slot][room] = (section, courses_map[course], teachers_map[professor])
                index += 8
    return timetable

def fitness_function(encoded):
    conflicts = 0
    professors_per_slot = {}
    index = 0
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            professors_per_slot.clear()
            for room in range(n_rooms):
                session = encoded[index:index+8]
                if session != '0' * 8:
                    section = int(session[:4], 2)
                    course = int(session[4:6], 2)
                    professor = int(session[6:8], 2)
                    if professor in professors_per_slot and professors_per_slot[professor] != room:
                        conflicts += 1
                    professors_per_slot[professor] = room
                index += 8
    return conflicts

def mutation(encoded, mutation_rate=0.02):
    mutant = list(encoded)
    for i in range(len(mutant)):
        if random.random() < mutation_rate:
            mutant[i] = '1' if mutant[i] == '0' else '0'
    return ''.join(mutant)

def crossover(parent1, parent2, crossover_rate=0.8):
    if random.random() < crossover_rate:
        pt = random.randint(1, len(parent1) - 1)
        return parent1[:pt] + parent2[pt:], parent2[:pt] + parent1[pt:]
    return parent1, parent2

def selection(population, scores, k=3):
    selected_index = random.randint(0, len(population) - 1)
    for _ in range(k-1):
        index = random.randint(0, len(population) - 1)
        if scores[index] < scores[selected_index]:
            selected_index = index
    return population[selected_index]

def genetic_algorithm(fitness_func, n_bits, n_iter, n_pop, crossover_rate, mutation_rate):
    population = [encode_timetable(generate_initial_timetable()) for _ in range(n_pop)]
    best, best_eval = None, float('inf')
    for gen in range(n_iter):
        scores = [fitness_func(individual) for individual in population]
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                print(">%d, new best score: %.3f" % (gen, best_eval))
        selected = [selection(population, scores) for _ in range(n_pop)]
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            for c in crossover(p1, p2, crossover_rate):
                mutated = mutation(c, mutation_rate)
                children.append(mutated)
        population = children
    return best, best_eval

n_bits = n_days * n_slots_per_day * n_rooms * 8
n_iter = 400
n_pop = 100
crossover_rate = 0.9
mutation_rate = 0.08

best_solution, best_score = genetic_algorithm(fitness_function, n_bits, n_iter, n_pop, crossover_rate, mutation_rate)
decoded_timetable = decode_timetable(best_solution)
print("Best Score:", best_score)

# Displaying the timetable in a formatted way
for day, day_name in days_map.items():
    print(f"\n{day_name}")
    for slot, time_name in times_map.items():
        print(f"\n{time_name}")
        for room in range(n_rooms):
            print(f"Room {room + 1}:", end=" ")
            for session in decoded_timetable[day][slot][room]:
                if session is None:
                    print("Empty", end=", ")
                else:
                    section, course, teacher = session
                    print(f"{sections_map[section]} - {course} ({teacher})", end=", ")
            print()


>0, new best score: 14.000
>0, new best score: 10.000
>0, new best score: 8.000
>0, new best score: 6.000
>0, new best score: 5.000
>0, new best score: 3.000
Best Score: 3

Monday

8:00 - 9:30
Room 1: 

TypeError: cannot unpack non-iterable int object